## Overview and Motivation  
<i>Provide an overview of the project goals and the motivation for it. Consider that this will be read by people who did not see your project proposal. </i>

We seek to collect and examine horse racing data in order to 

## Related Work 
<i>Anything that inspired you, such as a paper, a web site, or something we discussed in class.</i>

We first conducted a throough study of horse racing and betting therein as follows to understand what we were working with. 

## Initial Questions 
<i>What questions are you trying to answer? How did these questions evolve over the course of the project? What new questions did you consider in the course of your analysis? - Data: Source, scraping method, cleanup, storage, etc. </i>

## Final Analysis 
<i> What did you learn about the data? How did you answer the questions? How can you justify your answers? </i>

In [25]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests
import json
import time
import string
# from tqdm import *
import datetime
import math

## Scraping 
We began our adventure with an extensive amount of scraping.  We used many resources to access a large amount of data and hit many hiccups along the way.  For one, the data sources we were using were extremely varied and very disorganized.  

### Race History Scraping 
We obtained a significant amount of information on horse races across the US since 1998 from the website racingchannel.com.  The scraping process was complicated and long, but proved very useful.  

In [ ]:
# Gets main page's HTML
mainpagelink = 'http://racingchannel.com/results_archive.php'
mainpagehtml = requests.get(mainpagelink).text

We started at the mainpage of racingchannel.com (see below) to get a list of racetracks and racetrackcodes, and we then created a dict of the two, which was the basis for most of our subsequent scraping. This page on the website has hyperlinks to all the individual racetrack pages, so the first step of scraping was from here.  
<img src="images/image_1.png" width="600px"/>

In [ ]:
racetrackshtmllst = BeautifulSoup(mainpagehtml, 'html.parser').body.table.findAll('tr')[4].findAll('td')
tracks = []
trackcodes = []
trackdatalinks = []
for tracklst in racetrackshtmllst:
    tracklinks = tracklst.findAll('a')
    for link in tracklinks:
        trackdatalinks.append('http://racingchannel.com/'  + link.get('href').encode('ascii'))
        trackcodes.append(link.get('href')[10:13].encode('ascii'))
        tracks.append(link.text.encode('ascii'))
tracks[1] = 'Arlington Park'
trackcodedict = dict(zip(tracks, trackcodes))
tracklinkdict = dict(zip(tracks, trackdatalinks))
trackinfodf = pd.DataFrame(index=tracks)
trackinfodf['code'] = trackcodes
trackinfodf['link'] = trackdatalinks
trackinfodf.head()

Next, we then pulled the html of each individual track from the pages pulled above and beautifulSouped it.   The below code stores the cleaned html of all track info pages into a csv.  

In [ ]:
## WARNING: DO NOT RERUN THIS IF TEMPDATA ALREADY HAS TRACKINFO.CSV IN IT! ##

htmloftracks = []
for link in trackdatalinks:
    htmloftracks.append(BeautifulSoup(requests.get(link).text, 'html.parser'))
    print link
    time.sleep(5)

trackinfodf['track_html'] = htmloftracks
trackinfodf.head(10)

# Stores track codes and links in csv
trackinfodf.to_csv('tempdata/trackinfo.csv', index_label='name')

In [ ]:
# Loads trackinfodf from the csv in tempdata
try:
    del trackinfodf
except:
    pass
trackinfodf = pd.read_csv('tempdata/trackinfo.csv', index_col='name')
trackinfodf['track_html'] = trackinfodf['track_html'].apply(lambda h:BeautifulSoup(h, 'html.parser'))
trackinfodf.head(5)

Next, we stored all the years and months that races occured for each track.  From the above links, we are directed to a page of months and years, that each then direct to monthly calendars with races listed on the corresponding dates. Using this page, we scraped the needed years and months. A sample page for the track Aqueduct can be seen below. 

<img src="images/image_2.png" width="600px"/>

In [ ]:
# For each track rip the years and months that the track had races ran on it
tracksoups = trackinfodf['track_html']
yrmonthdicts_by_track = []
for soup in tracksoups:
    soup = soup.body.center.findAll('table')[-1].tr.findAll('td')
    years = []
    yrmonths_for_track = []
    for col in soup:
        year = col.text.encode('ascii')[:4]
        years.append(year)
        months = [elem.get('href').encode('ascii')[19:21] for elem in col.findAll('a')]
        yrmonths_for_track.append({year: months})

    yrmonthdicts_by_track.append(yrmonths_for_track)
biggie = dict(zip(trackcodes,yrmonthdicts_by_track))

Using the above obtained years and months, we next had to pull all the days from each calendar of races.  A sample month for October 1998 for the racetrack Aqueduct can be seen below.  
<img src="images/image_3.png" width="600px"/>

In [ ]:
# Scrapes all the days that the track was in operation for each month, year, and track combo
%time
trackday_lists = []
for track in sorted(biggie.keys()):
    for yeardict in biggie[track]:
        for month in yeardict.values()[0]:
            monthhtml = BeautifulSoup(requests.get('http://racingchannel.com/archives/{}/{}/{}/default.html'\
                                         .format(track,yeardict.keys()[0],month)).text, 'html.parser').body.center
            try:
                soup = monthhtml.findAll('table')[1]
            except:
                soup = monthhtml.findAll('table')[0]
            days = [daytag.get('href')[29:31].encode('ascii') for daytag in soup.findAll('a')]
            mo = daytag.get('href')[19:21].encode('ascii')
            yr = daytag.get('href')[14:18].encode('ascii')
            trk = daytag.get('href')[22:25].encode('ascii')
            for day in days:
                trackday_lists.append([trk, yr, mo, day]) 
            time.sleep(.00001)

    print track
    
print len(trackday_lists)

In [ ]:
# Stores a csv of the different days that races were run since 1998 at each track
# (scraped from the racingchannel.com website in order to make accessing the html easier)
trackdaylistdf = pd.DataFrame(trackday_lists)
trackdaylistdf.to_csv('tempdata/alldays_new.csv')

In [ ]:
# Reads back in the list of lists that contains all the days we need to request from racingchannel.com
bigdaydf = pd.read_csv('tempdata/alldays_new.csv')
bigdaydf = bigdaydf[bigdaydf.columns[-4:]]
bigdaydf.columns = ['track','year','month','day']
bigdaydf.head()

In [ ]:
# Ensure we have all the tracks
print tracks
bigdaydf.track.unique()

We made a decision to drop the international tracks for several reasons.  Firstly, we wanted to ensure the continuity of our data and there was spotty data from the international arenas.  Secondly, the weather data was hard to obtain for international locations, so it was dangerous.  Thirdly, considering the animal travel laws, it seemed difficult for international races to have significant impacts on our races as horses must be quarantined for extended periods of time whenever they cross international borders.  

In [ ]:
# Drop international tracks
bigdaydf = bigdaydf.set_index(bigdaydf['track'])
usadaydf = bigdaydf.drop(['GB1','ZA1','ZA2','AU1','AU2','FTE','HAS','NCF','WOB'])
print usadaydf.shape
print bigdaydf.shape
usadaydf.to_csv('tempdata/usadays_new.csv')

In [ ]:
# Read in the csv containing USA track days
usadaydf = pd.read_csv('tempdata/usadays_new.csv')
usadaydf.drop('track.1',axis=1, inplace=True)
print usadaydf.shape # this should be like 83000
print '\n'
usadaydf.head() #<--this is the one we want!!!!

This is where the most significant scraping began. Now that we had obtained the track names and corresponding race dates, we needed to scrape all the results for every date and every track. A sample results page for the Aqueduct track on October 28, 1998 can be seen below. 

<img src="images/image_4.png" width="600px"/>

This took a significant amount of time, so we included methods to track our progress along the way.  

In [ ]:
pretime = time.time()
superout = []
# Getting the raw HTML text from racingchannel to be later cleaned with BeautSoup
# The live 1-9 variables and the many if else statements that follow the main code all have to do with 
# setting and hitting checkpoints along the way
acc = 0
livee = True
live1 = True
live2 = True
live3 = True
live4 = True
live5 = True
live6 = True
live7 = True
live8 = True
live9 = True
for tup in usadaydf.itertuples():
    time.sleep(0.05)
    magiclink = 'https://www.racingchannel.com/archives/{}/{}/{:02}/{}{}{:02}{:02}.HTM'\
        .format(tup[1],tup[2],tup[3],tup[1],str(tup[2])[2:],tup[3],tup[4])
    htmlfrompage = requests.get(magiclink).text
    superout.append([tup[1],tup[2],tup[3],tup[4],htmlfrompage])
    acc += 1
    if acc > 100 and livee:
        livee = False
        print "Made it {}".format(acc)
    elif acc > 10000 and live1:
        live1 = False
        print "Made it {}".format(acc)
    elif acc > 20000 and live2:
        live2 = False
        print "Made it {}".format(acc)
    elif acc > 30000 and live3:
        live3 = False
        print "Made it {}".format(acc)
    elif acc > 40000 and live4:
        live4 = False
        print "Made it {}".format(acc)
    elif acc > 50000 and live5:
        live5 = False
        print "Made it {}".format(acc)
    elif acc > 60000 and live6:
        live6 = False
        print "Made it {}".format(acc)
    elif acc > 70000 and live7:
        live7 = False
        print "Made it {}".format(acc)
    elif acc > 80000 and live8:
        live8 = False
        print "Made it {}".format(acc)
        print "Home stretch!!!"  
    

superdf = pd.DataFrame(superout)
superdf.to_csv('tempdata/superdf.csv')

print time.time()-pretime, "seconds"

Our data obtained above was MASSIVE, so we had a very hard time reading the csv we created back into our iPython notebook. We constantly had memory errors when trying to read in the csv in one chunk, so used the chunksize option in pd.read_csv to overcome this problem.

In [ ]:
# adapted from http://stackoverflow.com/questions/17557074/memory-error-when-using-pandas-read-csv
csv_chunks = pd.read_csv('~/Desktop/superdf.csv', sep = ',', chunksize = 5000)
superdf = pd.concat(chunk for chunk in csv_chunks)
superdf = superdf[superdf.columns[-5:]]
superdf.columns = ['track','year','month','day','html']

In [ ]:
## When we saved the csv of a list, it saves it as a string.  This function then converts that 
## string back into a list. This turned out to be less useful than we thought when we wrote it.  
def stringtolist(string):
    elems = []
    curr = ''
    acc = 0
    for char in list(string):
        if char in '[':
            pass
        elif (char == ',' and list(string)[acc-1] in '>') or (char == ']'):
            elems.append(curr)
            curr = ''
        else:
            curr += char
        acc += 1
    return elems
a = '[<new>asdf</new>,<yeet>asdfaghfds,FF<yeet>,<FF>masdfasdf</FF>]'
print stringtolist(a)

In [ ]:
## We used a class object to make the scraping easier.  

class RaceDataSet(object):
    
    def __init__(self, num, datedata):
        self.num = num
        self.datedata = datedata
        self.first = None
        self.second = None
        self.third = None
        self.firstnum = None
        self.secondnum = None
        self.thirdnum = None
        self.win = None
        self.place1 = None
        self.place2 = None
        self.show1 = None
        self.show2 = None
        self.show3 = None
        
    def __str__(self):
        return "{} : {}.....{}.{}...{}.{}...{}.{}\n\t{}...{}.{}.....{}.{}.{}\n".format(self.datedata,self.num,self.firstnum,\
                    self.first,self.secondnum,self.second,self.thirdnum,self.third,self.win,self.place1,\
                    self.place2,self.show1,self.show2,self.show3)
    
    def to_list(self):
        return [self.datedata[0],self.datedata[1],self.datedata[2],self.datedata[3],self.num,self.firstnum,\
                    self.first,self.secondnum,self.second,self.thirdnum,self.third,self.win,self.place1,\
                    self.place2,self.show1,self.show2,self.show3]

We next created an important function that does a significant portion of the parsing of the html.  Firstly, it uses BeautifulSoup on the html, and then it goes through all of it, takes the row elements of each race, and puts the data into a dataframe.  This function is designed to take a row from superdf (the csv of the raw html), and then parse the html of only that page. Again, memory considerations forced us to approach the problem this way, as we could not hold very much of this data in memory at a time. Going row by row solved this problem.

In [ ]:
def extract(row):
    tags = None
    date = None
    nobolds = []
    if isinstance(row, pd.Series):
        tags = BeautifulSoup(row['html'], 'html.parser').html.body.findAll('table')
        date = [row['track'],row['year'],row['month'],row['day']]
    elif isinstance(row, tuple):
        if str(row[5]) == 'nan':
            return []
        tags = BeautifulSoup(row[5], 'html.parser').html.body.findAll('table')
        date = [row[1],row[2],row[3],row[4]]
    out = []
    for table in tags:
        bail = False
        if table.findAll('tr') == []:
            continue
        elif table.tr.findAll('b') == []:
            continue
        elif table.tr.td.b == None:
            nobolds.append((table.tr.td,date)) 
        elif 'race' in string.lower(table.tr.td.b.get_text().encode('ascii')):
            trows = table.findAll('tr')
            if len(trows) >= 4:
                row1 = trows[0].findAll('td')
                row2 = trows[1].findAll('td')
                row3 = trows[2].findAll('td')
                row4 = trows[3].findAll('td')
                if len(row2) > 0 and len(row3) > 0 and len(row4) > 0:
                    for elem in [row2[0].get_text().encode('ascii'),row3[0].get_text().encode('ascii'),\
                        row4[0].get_text().encode('ascii')]:
                        if '$2 D/Double' in str(elem):
                            bail = True
                    if not bail:
                        if not (len(row2) < 6 or len(row3) < 6 or len(row4) < 6):
                            racenum = row1[1].get_text().encode('ascii')
                            obj = RaceDataSet(racenum, date)
                            obj.firstnum = row2[1].get_text().encode('ascii')
                            obj.first = row2[2].get_text().encode('ascii')
                            obj.secondnum = row3[1].get_text().encode('ascii')
                            obj.second = row3[2].get_text().encode('ascii')
                            obj.thirdnum = row4[1].get_text().encode('ascii')
                            obj.third = row4[2].get_text().encode('ascii')
                            obj.win = row2[3].get_text().encode('ascii')
                            obj.place1 = row2[4].get_text().encode('ascii')
                            obj.place2 = row3[4].get_text().encode('ascii')
                            obj.show1 = row2[5].get_text().encode('ascii')
                            obj.show2 = row3[5].get_text().encode('ascii')
                            obj.show3 = row4[5].get_text().encode('ascii')

                            out.append(obj.to_list())
    return out

We then created a cleaned dataframe by running a for loop using the above function, going row by row through superdf. We focused on getting win, place, show payoffs for each horse.

In [ ]:
cleandf = []
count = 0
for tup in tqdm(superdf.itertuples()):
    count += 1
    #controls the scale of the problem during testing
    if count < 0:
        continue
    elif count > 1000000:
        break
    cleandf += extract(tup)
cleandf = pd.DataFrame(cleandf, columns=['track','year','month','day','racenum','firstnum','first','secondnum',\
                                         'second','thirdnum','third','win','place1','place2','show1','show2','show3'])
cleandf.to_csv('tempdata/cleandf.csv', columns=['track','year','month','day','racenum','firstnum','first','secondnum',\
                                         'second','thirdnum','third','win','place1','place2','show1','show2','show3'])
cleandf.head()

In [ ]:
cleandf = pd.read_csv('tempdata/cleandf.csv')
cleandf = cleandf[cleandf.columns[-17:]]
cleandf.head()

Our cleaned race history data is in the csv called cleandf.  This is what we will use when we run our analysis and investigation later on.  

### Weather Scraping 
With the race data in hand, we turned to obtaining the weather data for all relavent dates.  We were able to access the Wunderground weather API thanks to special permission from the Weather Channel.  Using this, we obtained weather conditions for every date that we had race information available for the corresponding zip codes.  

An example of how to use the API for a set date and zip code: 

In [ ]:
date = '20141225'
zip_code = '11420'
wunderground_url = 'http://api.wunderground.com/api/4a26cfc369eb7841/history_{}/q/{}.json'.format(date, zip_code)
examp = json.loads(requests.get(wunderground_url).text)

We decided on a list of weather conditions that would be most likely to effect the racetracks on any given day.  Given Wunderground had hundreds of weather parameters, we had to limit the scope of our scrape to avoid overfitting.  Many of the parameters given were duplicates, as each parameter was available in metric or imperial units, as well as max, min, and average. For the parameters we chose, we used metric units and average amounts if available and applicable.   

In [3]:
# weather metrics we want
weather_data = ['fog','hail','maxhumidity','meandewptm','meanpressurem','meantempm','meanvism',
                'meanwdird', 'meanwindspdm', 'precipm', 'rain', 'snow', 'snowdepthm','snowfallm', 'thunder',
                'minhumidity']

In [4]:
# function to format data returned from wunderground api to have only the metrics we want
def output_dict(in_dict):
    temp = [(elem,in_dict[elem]) for elem in in_dict.keys() if elem in weather_data]
    return dict(temp)

In [7]:
output_dict(examp['history']['dailysummary'][0])

{u'fog': u'1',
 u'hail': u'0',
 u'maxhumidity': u'96',
 u'meandewptm': u'3',
 u'meanpressurem': u'1008',
 u'meantempm': u'10',
 u'meanvism': u'13',
 u'meanwdird': u'261',
 u'meanwindspdm': u'27',
 u'minhumidity': u'35',
 u'precipm': u'1.52',
 u'rain': u'1',
 u'snow': u'0',
 u'snowdepthm': u'0.00',
 u'snowfallm': u'0.00',
 u'thunder': u'0'}

In [8]:
# read in the csv of dates we want to get weather data for from previous scraping 
datesdf = pd.read_csv("tempdata/usadays.csv")
datesdf.drop('track.1',axis=1, inplace=True)
datesdf.head()

,track,year,month,day
0,AQU,1998,10,28
1,AQU,1998,10,29
2,AQU,1998,10,30
3,AQU,1998,10,31
4,AQU,1998,11,1


In [ ]:
# convert date components to strings and add a 0 before single digit month/days
datesdf[['year', 'month', 'day']] = datesdf[['year', 'month', 'day']].astype(str)
for line in datesdf.index:
    if len(datesdf.loc[line]['month']) == 1:
        datesdf.loc[line]['month'] = '0' + datesdf.loc[line]['month']
    if len(datesdf.loc[line]['day']) == 1:
        datesdf.loc[line]['day'] = '0' + datesdf.loc[line]['day']

# dictionary keyed by track to store corresponding date strings
dates_dict = {}

# stores all dates on which races occurred for a given track identifier
for track in datesdf.track.unique():
    datestring = []
    for row in datesdf.index:
        if datesdf.iloc[row]['track'] == track:
            datestring.append(str(datesdf.iloc[row]['year']) + str(datesdf.iloc[row]['month']) + 
                              str(datesdf.iloc[row]['day']))
    dates_dict[track] = datestring 
    
# zipcodes for all U.S. tracks, looked up manually and arranged into alphabetical order (according to 
# track abbreviations)
zips1 = ['11420', '60005','94403', '11003', '43123', '33056', '55379','40208', '23124', '25438', '70668', 
         '92014', '19804', '42420', '98001', '70570', '70119', '14425', '91768', '62234', '49415', '94710', 
         '33009', '60804', '46013', '90305', '46176', '40510', '42134', '20725', '71111', '75050', '90720', 
         '07073', '07757', '26047', '71901', '85023', '17028', '19020', '21215', '50009', '73111', '78154', 
         '45230', '77064', '91007', '12866', '60804', '02128', '33626', '44128', '21093', '41042']

# obtained by manual lookup, arranged in alphabetical order (according to 3 letter track identifier)
locs = datesdf.track.unique()

# dictionary mapping track identifiers to zipcodes
zips_dict1 = dict(zip(locs, zips1))

This is the main scraping function for the weather data.  We went by zip code and date and pulled the entire html for each location and time.  

In [ ]:
# queries wunderground API for every track-date combination
# stores results in dictionary keyed by (track id, date) tuple
%%time
weather_dict = {}
except_list = []
for key in dates_dict.keys():
    for fdate in dates_dict[key]:
        wunderground_url = 'http://api.wunderground.com/api/4a26cfc369eb7841/history_{}/q/{}.json'.format(fdate, zips_dict1[key])
        try:
            temp = json.loads(requests.get(wunderground_url).text)['history']['dailysummary'][0]
            weather_dict[(key, fdate)] = output_dict(temp)
        except:
            except_list.append((key,fdate))

In [ ]:
## WARNING: do NOT run this line again ## 
# weather_df.to_csv('tempdata/weather.csv')

We were able to save our data into a csv by running all the scraping and cleaning commands above, which gave us the following dataframe.  

In [4]:
weather_df = pd.read_csv("tempdata/weather.csv", index_col=0)
weather_df.head()

,track,date,fog,hail,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
0,REM,2004-09-25,0,0,94,12,1021,21,11,13,8,23,0.00,0,0,NaN,0,0
1,DPK,2013-06-12,0,0,87,17,1010,24,16,299,14,47,0.00,0,0,0,0,0
2,FMT,2013-06-01,0,0,94,17,1009,22,15,182,10,61,12.95,1,0,NaN,0,1
3,PAR,2005-02-25,0,0,80,6,1012,13,16,30,8,38,0.00,0,0,NaN,0,0
4,BEU,2013-05-04,0,0,66,6,1017,17,16,101,16,40,0.00,0,0,NaN,0,0


### Cleaning the Weather Data  

In [ ]:
# converts all cells in date column to datetime objects
weather_df.date = weather_df.date.apply(lambda x: datetime.datetime(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))

In [ ]:
# handles non-numberic entries in numberic columns
weather_df.precipm.loc[weather_df.precipm =="T"] = np.nan
weather_df.snowdepthm.loc[weather_df.snowdepthm == "T"] = np.nan
weather_df.snowfallm.loc[weather_df.snowfallm == "T"] = np.nan

# casts all weather metrics as floats
for col in weather_df.iloc[:,2:]:
    weather_df[col] = weather_df[col].astype(float)
weather_df.head()

In [ ]:
weather_df.to_csv('tempdata/weather.csv')

### Test Data Scraping
We obtained the full race results for the three test races of interest: The Kentucky Derby, The Preakness Downs, and the Belmont Stakes.  We obtained all this data from many Wikipedia pages, which proved very frustrating, as they were atrociously messy.  We ran into many corner cases, so that took a lot of investigation to account for.  

In [ ]:
## we create any empty dictionary which will hold all the html from the wikipedia pages 
pages = {}

## we create a dictionary of track years linked to the track names for the races 
## Belmont Stakes only had data from 2006 onward, while for the other two, we got data from 
## 1998 onward as that is how far our training data goes 
years1 = [str(i) for i in range(1998,2016)]
years2 = [str(i) for i in range(2006,2016)]
track_year_dict = {"_Kentucky_Derby": years1, "_Preakness_Stakes": years1, "_Belmont_Stakes": years2}

## obtaining all the html pages and putting them into our dictionary pages 
for key in track_year_dict.keys():
    for year in track_year_dict[key]:
        pages[year+key] = requests.get("https://en.wikipedia.org/wiki/{}".format(year+key)).text
        time.sleep(0.1)
        
# function to parse scraping output
# returns 2 data frames, one for payouts and one for results for a given race in a given year
def parser(key, page_dict):
    soup = BeautifulSoup(page_dict[key], "html.parser")
    tables = soup.find_all("table", attrs={"class": "wikitable"})
    
    if len(tables[0].find_all("tr")) <= 5:
        table1 = tables[0].find_all("tr")
        table2 = tables[1].find_all("tr")
    else:
        table1 = tables[1].find_all("tr")
        table2 = tables[0].find_all("tr")
    
    t1headers = [elem.get_text() for elem in table1[0].find_all("th")]
    t2headers = [elem.get_text() for elem in table2[0].find_all("th")]
    if (key == "2005_Kentucky_Derby"):
        t2headers.append("Time")
        t2headers[t2headers.index("Jockey")] = "Horse"
    
    t1 = []
    t2 = []
    for row1 in table1[1:]:
        r1_data = [cell.get_text() for cell in row1.find_all("td")]
        t1.append(r1_data)
    for row2 in table2[1:]:
        # handles cases where cells in horse column all have header tags
        if row2.find("th"):
            r2_data = [cell.get_text() for cell in row2.find_all("td")]
            r2_data.insert(2, row2.find("th").get_text())
            t2.append(r2_data)
        else:
            r2_data = [cell.get_text() for cell in row2.find_all("td")]
            t2.append(r2_data)       
    try:
        payout = pd.DataFrame(t1, columns=t1headers)
        results = pd.DataFrame(t2, columns=t2headers)
    except Exception,e:
        # handles 2015 Kentucky Derby results table, which doesn't have a header row
        if key == "2015_Kentucky_Derby":
            t1headers = [elem.get_text() for elem in table1[0].find_all("td")]
            payout = pd.DataFrame(t1, columns=t1headers)
            results = pd.DataFrame(t2, columns=t2headers)
        else:
            print str(e)
  
    return (payout, results)

# dictionary of data frames keyed by track-year string
# values are tuples of data frames returned by parser
bigdict = {key:parser(key, pages) for key in pages.keys()}

# constructs single payouts data frame by concatenating all payout data frames contained in bigdict
payouts_df = pd.DataFrame(columns=["Post", "Horse", "Win", "Place", "Show", "Track", "Year"])
for track in track_year_dict.keys():
    for year in track_year_dict[track]:
        access = year+track
        bigdict[access][0].columns = ["Post", "Horse", "Win", "Place", "Show"]
        bigdict[access][0]["Track"] = track
        bigdict[access][0]["Year"] = year
        payouts_df = pd.concat([payouts_df, bigdict[access][0]], ignore_index = True)

In [9]:
payouts_df = pd.read_csv("tempdata/payouts.csv", index_col=0)
payouts_df.head()

,Post,Horse,Win,Place,Show,Track,Year
0,11,Real Quiet,$7.00,$3.60,$3.00,_Preakness_Stakes,1998
1,10,Victory Gallop,-,$3.20,$2.80,_Preakness_Stakes,1998
2,3,Classic Cat,-,-,$4.80,_Preakness_Stakes,1998
3,6,Charismatic,$18.80,$7.60,$5.80,_Preakness_Stakes,1999
4,5,Menifee,-,$3.60,$3.20,_Preakness_Stakes,1999


In [10]:
# function to clean "Win", "Place", and "Show" columns above
# converts strings to ints
def dollar_clean(inpt):
    try:
        if "$" in inpt:
            a = inpt.split("$")
            a = a[1]
            if " " in a:
                b = a.split(" ")
                c = b[1]
                return float(c)
            else:
                return float(a)
        else:
            return float(inpt)
    except:
        return inpt
    
# function to handle non-numeric entries in "Post" column above
def clean_post(inpt):
    try:
        return int(inpt)
    except:
        return 0

In [11]:
# clean payouts df
payouts_df.replace("-", np.nan, inplace=True)
payouts_df.replace(u"\u2013", np.nan, inplace=True)
payouts_df.replace("", np.nan, inplace=True)
payouts_df.Post = payouts_df.Post.apply(clean_post)
payouts_df.Win = payouts_df.Win.apply(dollar_clean)
payouts_df.Place = payouts_df.Place.apply(dollar_clean)
payouts_df.Show = payouts_df.Show.apply(dollar_clean)
payouts_df.head()

,Post,Horse,Win,Place,Show,Track,Year
0,11,Real Quiet,7.0,3.6,3.0,_Preakness_Stakes,1998
1,10,Victory Gallop,NaN,3.2,2.8,_Preakness_Stakes,1998
2,3,Classic Cat,NaN,NaN,4.8,_Preakness_Stakes,1998
3,6,Charismatic,18.8,7.6,5.8,_Preakness_Stakes,1999
4,5,Menifee,NaN,3.6,3.2,_Preakness_Stakes,1999


In [ ]:
def any_in(my_list, comp_list):
    ret_list = []
    for item in my_list:
        if item in comp_list:
            ret_list.append(item)
    return ret_list

def dropper(df, my_list):
    drops = any_in(my_list, df.columns)
    df.drop(drops, axis=1, inplace=True)
    return df

def renamer(df, old_names, new_name):
    if any_in(old_names, df.columns):
        rename = any_in(old_names, df.columns)[0]
        newcolnms = list(df.columns)
        index = newcolnms.index(rename)
        newcolnms[index] = new_name
        df.columns = newcolnms
    
    
drop_list = ["Trainer", "Jockey", "Owner", "Lengths\nBehind", "Time / behind", "Time/\nBehind", 
             "Trainer\n\n", "Points[7]", "Points", "Previous Race", "Time"]
Horse = ["Horse name"]
Finish = ["Finish\nPosition", "Finish[2]", "Results", "Finished", "Position", "Finish\nposition", "Finish"]
Post = ["Post\nPosition", "Number", "Program\nNumber", "Post\nposition", "Pgm", "Post"]
Stakes = ["Stake", "Purse\nEarnings", "Stakes"]
MOdds = ["Morning Line\nOdds", "Morning\nline\nodds", "Opening odds (to 1)", "Opening Odds", "Morning\nodds[3]",
        "Morning Line Odds", "Morning Line\nodds", "Morning\nOdds[3]", "Opening\nOdds"]
POdds = ["Final\nodds[1]", "Post Time\nOdds", "Final odds[4]", "Post Time\nodds", "Final Odds", "Final Odds[2]",
        "Final odds", "Final odds\nto $1[1]", "Odds"]

for k in bigdict.keys():
    dropper(bigdict[k][1], drop_list)
    renamer(bigdict[k][1], Horse, "Horse")
    renamer(bigdict[k][1], Finish, "Finish")
    renamer(bigdict[k][1], Post, "Post")
    renamer(bigdict[k][1], Stakes, "Stakes")
    renamer(bigdict[k][1], MOdds, "Morning Line Odds")
    renamer(bigdict[k][1], POdds, "Post Time Odds")

In [ ]:
results_df = pd.DataFrame(columns=["Track", "Year", "Finish", "Post", "Horse", "Morning Line Odds", "Post Time Odds", "Stakes"])
for track in track_year_dict.keys():
    for year in track_year_dict[track]:
        access = year+track
        bigdict[access][1]['Track'] = track
        bigdict[access][1]['Year'] = year
        results_df = pd.concat([results_df, bigdict[access][1]], ignore_index = True)

In [ ]:
for c in ["Morning Line Odds", "Post Time Odds"]:
    for i in results_df.loc[(results_df.Track == "_Belmont_Stakes")&(results_df.Year == "2015")].index:
        results_df[c].iloc[i] = results_df[c].iloc[i][20:]
results_df.head()

In [ ]:
# http://stackoverflow.com/questions/944700/how-to-check-for-nan-in-python
def isNaN(num):
    return num != num

def make_odds(item):
    try:
        if (item == "scr") or (item == "Scr") or (item == "SCR") or (item == "DNR"):
            return 0
        elif " !" in item: 
            return float(item[0:4])
        
        q = item.split(u'\u2660')
        if len(q) > 1:
            q = q[1]
        else:
            q = q[0]
        
        y = q.split("\n")
        if "-" in y[0]: 
            x = y[0].split("-")
        elif u"\u2014" in y[0]:
            x = y[0].split(u"\u2014")
        if len(x) > 1:
            return float(x[0])/float(x[1])
        else:
            
            return float(a[0])
    except Exception, e:
        return item

def clean_finish(inpt):
    if not isinstance(inpt, int):
        if len(inpt) > 2 and inpt[:-2].isdigit():
            return int(inpt[:-2])
        elif inpt.isdigit():
            return int(inpt)
        else:
            return 0
    else:
        return inpt

In [ ]:
smallresults_df = results_df[["Finish", "Post", "Horse", "Morning Line Odds", "Post Time Odds", "Track", "Year"]]
smallresults_df["Odds"] = np.nan
for num in smallresults_df.index:
    if not isNaN(smallresults_df["Morning Line Odds"].iloc[num]):
        smallresults_df["Odds"].iloc[num] = smallresults_df["Morning Line Odds"].iloc[num]
    else:
        smallresults_df["Odds"].iloc[num] = smallresults_df["Post Time Odds"].iloc[num]
smallresults_df.drop(["Morning Line Odds", "Post Time Odds"], axis=1, inplace=True)
smallresults_df["Odds2"] = smallresults_df.Odds.apply(make_odds)
smallresults_df.head()

In [ ]:
#smallresults_df.to_csv('tempdata/smallresults.csv', encoding="utf8")

In [ ]:
smallresults_clean = pd.read_csv("tempdata/109_OddsDF.csv", index_col=0)
smallresults_clean["Odds"] = smallresults_clean["Odds2"]
smallresults_clean.drop("Odds2", axis=1, inplace=True)
smallresults_clean = smallresults_clean.reset_index()
smallresults_clean.drop("index", axis=1, inplace=True)
smallresults_clean.Horse.iloc[190] = smallresults_clean.Horse.iloc[190][:-3]
smallresults_clean.Post.iloc[528:540] = [8,9,2,11,6,3,1,4,12,5,7,10]
smallresults_clean.Finish.iloc[557:562] = [1,2,3,4,5]
smallresults_clean.Finish.iloc[615:617] = [4,4]
smallresults_clean.Finish = smallresults_clean.Finish.apply(clean_finish)
smallresults_clean.Post = smallresults_clean.Post.apply(clean_post)
smallresults_clean.Year = smallresults_clean.Year.astype(str)

In [ ]:
smallresults_clean.to_csv("tempdata/smallresults_clean.csv")

In [ ]:
result_testdf = pd.merge(payouts_df, smallresults_clean, how="right", on=["Track", "Year", "Horse", "Post"])

In [ ]:
result_testdf.to_csv("tempdata/result_testdf.csv")

### Cleaning up above data to make it the format we want 

In [45]:
cleandf = pd.read_csv("tempdata/cleandf.csv", index_col=0); 
cleandf.head() 

,track,year,month,day,racenum,firstnum,first,secondnum,second,thirdnum,third,win,place1,place2,show1,show2,show3
0,AQU,1998,10,28,1,3,CURVY IMAGE,6,DYNABILITY,2,ALYDAR'S JET,6.3,3.8,5.9,3.0,4.3,5.0
1,AQU,1998,10,28,2,3,MILLIONDOLLARSMILE,8,THIS BLISS,2,DOUBLE COVERAGE,14.0,4.8,3.4,3.1,2.6,3.0
2,AQU,1998,10,28,3,6,ROOK,2,ALEXTHETHIRD,8,LITTLE GHAZI,6.5,3.4,3.5,2.4,2.2,2.2
3,AQU,1998,10,28,4,2,WAPO,4,AUNT EVIE,8,WINTER BID,6.2,5.1,67.0,2.8,11.6,2.1
4,AQU,1998,10,28,5,8,FUNNY TOY (ARG),6,KANE KATI,1,MT. ARROW,16.0,7.7,6.3,3.5,3.9,3.1


In [46]:
cleandf["date"] = [datetime.datetime(cleandf.year.iloc[row],cleandf.month.iloc[row],cleandf.day.iloc[row]) for row in cleandf.index]; 
cleandf.drop(["year", "month", "day"], inplace = True, axis =1 ); 

subdf1 = cleandf[["track", "racenum", "date", "firstnum", "first", "win", "place1", "place2", "show1", "show2", "show3"]]; 
subdf2 = cleandf[["track", "racenum", "date", "secondnum", "second", "win", "place1", "place2", "show1", "show2", "show3"]]; 
subdf3 = cleandf[["track", "racenum", "date", "thirdnum", "third", "win", "place1", "place2", "show1", "show2", "show3"]]; 
subdf1["fin_pos"] = 1; 
subdf2["fin_pos"] = 2; 
subdf3["fin_pos"] = 3; 

subdf1.rename(columns={'firstnum': 'post', 'first': 'name'}, inplace=True); 
subdf2.rename(columns={'secondnum': 'post', 'second': 'name'}, inplace=True); 
subdf3.rename(columns={'thirdnum': 'post', 'third': 'name'}, inplace=True); 

subdf3.win = 0; 
subdf3["place"] = 0;  
subdf3["show"] = subdf3.show3; 

subdf2.win = 0; 
subdf2["place"] = subdf2.place2;  
subdf2["show"] = subdf2.show2; 

subdf1["place"] = subdf1.place1;  
subdf1["show"] = subdf1.show1;  

subdf1.drop(["place1", "place2", "show1", "show2", "show3"], inplace = True, axis =1 ); 
subdf2.drop(["place1", "place2", "show1", "show2", "show3"], inplace = True, axis =1 ); 
subdf3.drop(["place1", "place2", "show1", "show2", "show3"], inplace = True, axis =1 ); 

frames = [subdf1, subdf2, subdf3]; 
final_race_df = pd.concat(frames); 

/Users/morganfbreitmeyer/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/morganfbreitmeyer/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/morganfbreitmeyer/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in th

In [47]:
final_race_df.head(25)

,track,racenum,date,post,name,win,fin_pos,place,show
0,AQU,1,1998-10-28,3,CURVY IMAGE,6.3,1,3.8,3.0
1,AQU,2,1998-10-28,3,MILLIONDOLLARSMILE,14.0,1,4.8,3.1
2,AQU,3,1998-10-28,6,ROOK,6.5,1,3.4,2.4
3,AQU,4,1998-10-28,2,WAPO,6.2,1,5.1,2.8
4,AQU,5,1998-10-28,8,FUNNY TOY (ARG),16.0,1,7.7,3.5
5,AQU,6,1998-10-28,6,RUBY RUBLES,11.4,1,5.6,4.8
6,AQU,7,1998-10-28,5,EXCHANGE PLACE,9.2,1,4.4,2.9
7,AQU,8,1998-10-28,2,EARLY WARNING,16.4,1,6.6,5.3
8,AQU,9,1998-10-28,4,JET BLACK,20.4,1,11.0,6.4
9,AQU,1,1998-10-29,1a,CRITICS ACCLAIM,3.6,1,2.2,2.1


In [66]:
weather_df = pd.read_csv("tempdata/weather.csv", index_col=0);
weather_df.date = pd.to_datetime(weather_df.date)
weather_df.head()

,track,date,fog,hail,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
0,REM,2004-09-25,0,0,94,12,1021,21,11,13,8,23,0.00,0,0,NaN,0,0
1,DPK,2013-06-12,0,0,87,17,1010,24,16,299,14,47,0.00,0,0,0,0,0
2,FMT,2013-06-01,0,0,94,17,1009,22,15,182,10,61,12.95,1,0,NaN,0,1
3,PAR,2005-02-25,0,0,80,6,1012,13,16,30,8,38,0.00,0,0,NaN,0,0
4,BEU,2013-05-04,0,0,66,6,1017,17,16,101,16,40,0.00,0,0,NaN,0,0


In [ ]:
## final_race_df.to_csv('tempdata/final_race_df.csv', index_label='name')

In [86]:
## rereading the horse data in from my desktop because too big to store in github 
final_race_df = pd.read_csv("/Users/morganfbreitmeyer/Desktop/final_race_df.csv", index_col=0);
final_race_df.date = pd.to_datetime(final_race_df.date)

In [93]:
## merging the weather for all past races with the race results into our training data 
traindf = pd.merge(final_race_df, weather_df, on = ["track", "date"], how = "left")
traindf.rename(columns={'name.1': 'name'}, inplace=True); 
traindf.head()

,track,racenum,date,post,name,win,fin_pos,place,show,fog,hail,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
0,AQU,1,1998-10-28,3,CURVY IMAGE,6.3,1,3.8,3.0,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
1,AQU,2,1998-10-28,3,MILLIONDOLLARSMILE,14.0,1,4.8,3.1,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
2,AQU,3,1998-10-28,6,ROOK,6.5,1,3.4,2.4,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
3,AQU,4,1998-10-28,2,WAPO,6.2,1,5.1,2.8,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0
4,AQU,5,1998-10-28,8,FUNNY TOY (ARG),16.0,1,7.7,3.5,0,0,96,11,1013,12,15,195,13,75,4.06,1,0,NaN,0,0


In [11]:
traindf.shape

(2340444, 25)

In [ ]:
traindf.to_csv("/Users/morganfbreitmeyer/Desktop/traindf.csv")

In [67]:
## we had to manually input the dates for the testdata
preakness_dates = {1998:datetime.datetime(1998, 5, 16), 1999:datetime.datetime(1999, 5, 15),  
                   2000:datetime.datetime(2000, 5, 20), 2001:datetime.datetime(2001, 5, 19), 
                   2002:datetime.datetime(2002, 5, 18), 2003:datetime.datetime(2003, 5, 17),
                   2004:datetime.datetime(2004, 5, 15), 2005:datetime.datetime(2005, 5, 21),
                   2006:datetime.datetime(2006, 5, 20), 2007:datetime.datetime(2007, 5, 19),
                   2008:datetime.datetime(2008, 5, 17), 2009:datetime.datetime(2009, 5, 16),
                   2010:datetime.datetime(2010, 5, 15), 2011:datetime.datetime(2011, 5, 21),
                   2012:datetime.datetime(2012, 5, 19), 2013:datetime.datetime(2013, 5, 18),
                   2014:datetime.datetime(2014, 5, 17), 2015:datetime.datetime(2015, 5, 16)}

belmont_dates = {2006:datetime.datetime(2006, 6, 10), 2007:datetime.datetime(2007, 6, 9), 
                 2008:datetime.datetime(2008, 6, 7), 2009:datetime.datetime(2009, 6, 6), 
                 2010:datetime.datetime(2010, 6, 5), 2011:datetime.datetime(2011, 6, 1),
                 2012:datetime.datetime(2012, 6, 9), 2013:datetime.datetime(2013, 6, 8),
                 2014:datetime.datetime(2014, 6, 7), 2015:datetime.datetime(2015, 6, 6)}

kentucky_dates = {1998: datetime.datetime(1998, 5, 2), 1999:datetime.datetime(1999, 5, 1),  
                   2000:datetime.datetime(2000, 5, 6), 2001:datetime.datetime(2001, 5, 5), 
                   2002:datetime.datetime(2002, 5, 4), 2003:datetime.datetime(2003, 5, 3),
                   2004:datetime.datetime(2004, 5, 1), 2005:datetime.datetime(2005, 5, 7),
                   2006:datetime.datetime(2006, 5, 6), 2007:datetime.datetime(2007, 5, 5),
                   2008:datetime.datetime(2008, 5, 3), 2009:datetime.datetime(2009, 5, 2),
                   2010:datetime.datetime(2010, 5, 1), 2011:datetime.datetime(2011, 5, 7),
                   2012:datetime.datetime(2012, 5, 5), 2013:datetime.datetime(2013, 5, 4),
                   2014:datetime.datetime(2014, 5, 3), 2015:datetime.datetime(2015, 5, 2)}

In [95]:
## matching up all the dates for the test data with the data set and merging with weather to 
## create our final test data set 
final_df = pd.read_csv('tempdata/result_testdf.csv', index_col=0);
final_df.rename(columns = {'Finish':'fin_pos', 'Horse':'name', 'Post':'post', 'Win':'win', 'Place':'place', 'Show':'show'}, inplace = True)
datelist = [] 
tracklist = [] 

for row in final_df.index: 
    if final_df.iloc[row].Track == "_Preakness_Stakes":  
        datelist.append(preakness_dates[final_df.iloc[row].Year])
        tracklist.append("PIM")
    elif final_df.iloc[row].Track == "_Belmont_Stakes": 
        datelist.append(belmont_dates[final_df.iloc[row].Year])
        tracklist.append("BEL")
    elif final_df.iloc[row].Track == "_Kentucky_Derby": 
        datelist.append(kentucky_dates[final_df.iloc[row].Year])
        tracklist.append("CHU")

final_df["date"] = datelist
final_df["track"] = tracklist 

final_df.drop(["Year", "Track"], inplace = True, axis = 1); 
final_df

## now merging with the Weather data 
testdf = pd.merge(final_df, weather_df, on = ["track", "date"], how = "left")
testdf
testdf.to_csv("tempdata/testdf.csv")

In [96]:
testdf.shape

(631, 25)

In [49]:
#############################################################################
## READ IN THE CSVs WITH ALL THE DATA AND REDO THE DATETYPE TRANSFORMATION ## 
#############################################################################
traindf = pd.read_csv('/Users/George/DropBox/CS 109/traindf.csv', index_col=0);
traindf.date = pd.to_datetime(traindf.date)
traindf['name'] = traindf['name'].astype(str)
traindf['name'] = map(lambda x: x.upper(), traindf['name'])
traindf.loc[traindf['win']=='NaN'] = 0.0
traindf.loc[traindf['place']=='NaN'] = 0.0
traindf.loc[traindf['show']=='NaN'] = 0.0
traindf['win'].fillna(0, inplace=True)

testdf = pd.read_csv('tempdata/testdf.csv', index_col=0);
testdf.date = pd.to_datetime(testdf.date)
testdf['name'] = testdf['name'].astype(str)
testdf['name'] = map(lambda x: x.upper(), testdf['name'])
testdf[['win', 'place', 'show']].fillna(0, inplace=True)

In [50]:
traindf = traindf[traindf.name != 'NAN']
traindf.reset_index().drop('index', axis=1, inplace=True);

In [49]:
# Just checking for newlines since we thought we saw one
for i in range(len(traindf)):
    if '\n' in traindf.iloc[i]['name']:
        print 'newline'

In [51]:
testdf.head()

,post,name,win,place,show,fin_pos,Odds,date,track,fog,hail,maxhumidity,meandewptm,meanpressurem,meantempm,meanvism,meanwdird,meanwindspdm,minhumidity,precipm,rain,snow,snowdepthm,snowfallm,thunder
0,11,REAL QUIET,7.0,3.6,3.0,1,2.5,1998-05-16,PIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,VICTORY GALLOP,NaN,3.2,2.8,2,2.0,1998-05-16,PIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,CLASSIC CAT,NaN,NaN,4.8,3,12.3,1998-05-16,PIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,CHARISMATIC,18.8,7.6,5.8,1,8.4,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0,0,0,NaN,0,0
4,5,MENIFEE,NaN,3.6,3.2,2,2.0,1999-05-15,PIM,0,0,64,2,1025,16,16,75,14,24,0,0,0,NaN,0,0


# Working on a Baseline

In [6]:
def find_races(horses_name, traindf=traindf):
    return traindf[traindf['name']==horses_name]

def find_avg_payouts(df, race_date=datetime.date.today()):
    out = {}
    df = df[df.date < race_date]
    out['win'] = np.mean(df.win[df.win != 0.0])
    out['place'] = np.mean(df.place[df.place != 0.0])
    out['show'] = np.mean(df.show[df.show != 0.0])
    return out

find_avg_payouts(find_races('SMARTY JONES', traindf=testdf), race_date=datetime.datetime(1998, 12, 12))

{'place': nan, 'show': nan, 'win': nan}

The payouts are already standardized to $2 bets, so we luckily don't have to deal with that problem

In [34]:
unique_test_dates = testdf['date'].unique()

def find_horses(date):
    return testdf['name'].loc[testdf['date'] == date]

def find_track(date):
    return testdf['track'].loc[testdf['date'] == date][0]

#find_horses(datetime.datetime(1998, 5, 15))
#type(testdf['date'][0])

We run a simple regression (aka Average) to get a dict where the keys are a tuple `(date, track, horse)` and the values are themselves dictionaries with keys `win`, `place`, `show` and values as the corresponding averages leading into raceday (excluding information we couldn't have on raceday). 

In [11]:
avg_payouts = {}
for i in testdf['date'].unique():
    race = testdf.loc[testdf.date == i]
    for j in range(len(race)):
        date = race.date.iloc[j]
        track = race.track.iloc[j]
        horse = race.name.iloc[j]
        avg_payouts[(date, track, horse)] = find_avg_payouts(
            find_races(race['name'].iloc[j], traindf=traindf), race_date=date)

In [ ]:
## this function takes a payout and returns the first odds digit, assuming a comparison to 1, i.e. x-1 
## of that horse to win when betting occured assuming original bet of $2 and assumed take by race track of 15% 
def payoff_to_odds(payoff, bet_amount=2.0, take = .15):
    return round(((payoff/(1-take) - bet_amount)/bet_amount),4)

def odds_to_percent(odds): 
    return (float(str(odds)[2]))/(float(str(odds)[0])+float((str(odds)[2])))

def normalize_odds(odds): 
    x = odds.split("-")
    if len(x) > 1: 
        return float(x[0])/float(x[1])
    else: 
        return float(x[0])
    
def make_favorite(string): 
    if "favorite" in string: 
        return True 
    else:
        return False 

In [33]:
def baseline(indict):
    profit = []
    cost = []
    count = []
    count2 = []
    for key in indict.keys():
        count2.append(1)
        date = key[0]
        track = key[1]
        horse = key[2]
        
        exp_win = indict[key]['win']
        exp_place = indict[key]['place']
        exp_show = indict[key]['show']
        
        # get the morning line odds for that day to compare
        odds = testdf.loc[(testdf.name == horse) & (testdf.track == track) & (testdf.date == date)]['Odds'].iloc[0]

        if odds > payoff_to_odds(exp_win):
            cost.append(2)
            
            if math.isnan(testdf.loc[(testdf.name == horse) & 
                                     (testdf.track == track) & 
                                     (testdf.date == date)]['win'].iloc[0]):
                count.append(1)
                pass
            
            else:
                profit.append(testdf.loc[(testdf.name == horse) & 
                                     (testdf.track == track) & 
                                     (testdf.date == date)]['win'].iloc[0])
        else:
            count.append(1)
            pass
        
    return ['baseline', no_bets_placed, race, year, profit_wins, profit_places, profit_shows, total_profit, cost]
            
baseline(avg_payouts)

(-320.20000000000005, 625.79999999999995, 946)

Dataframe of race, year, profit/loss on win baseline, profit/loss on place baseline, profit/loss on show baseline

In [46]:
def make_bets_baseline(indict, unique_test_dates=unique_test_dates): 
    outdict = {}
    outdict['method'] = []
    outdict['no_bets_placed'] = []
    outdict['race'] = []
    outdict['year'] = []
    outdict['profit_wins'] = []
    outdict['profit_places'] = []
    outdict['profit_shows'] = []
    outdict['total_profit'] = []
    outdict['cost'] = []
    for unique_date in unique_test_dates:
        newdict = {}
        for key in indict.keys():
            date = key[0]
            track = key[1]
            horse = key[2]
            
            
            if date == unique_date:
                newdict[key] = indict[key]
                  
        # now we have newdict, subsetting based on race (by unique date)
        
        spread_tup = []
        for key in newdict.keys():
            date = key[0]
            track = key[1]
            horse = key[2]

            exp_win = newdict[key]['win']
            exp_place = newdict[key]['place']
            exp_show = newdict[key]['show']

            # get the morning line odds for that day to compare
            odds = testdf.loc[(testdf.name == horse) & 
                              (testdf.track == track) & 
                              (testdf.date == date)]['Odds'].iloc[0]
            
            spread = odds - payoff_to_odds(exp_win)
            spread_tup.append((horse, spread))
            
        sorted_list = sorted(spread_tup,key=lambda x: x[1], reverse=True)
        
        print sorted_list[0:3]
            
#         outdict['method'] = []
#         outdict['no_bets_placed'] = []
#         outdict['race'] = []
#         outdict['year'] = []
#         outdict['profit_wins'] = []
#         outdict['profit_places'] = []
#         outdict['profit_shows'] = []
#         outdict['total_profit'] = []
#         outdict['cost'] = []

make_bets_baseline(avg_payouts)

[('HOT WELLS', nan), ('REAL QUIET', nan), ("CORONADO'S QUEST", nan)]
[('WORLDLY MANNER', nan), ('ADONIS', nan), ('BADGE', 55.0)]
[('HUGH HEFNER', 45.111800000000002), ('RED BULLET', nan), ("HAL'S HOPE", 18.805900000000001)]
[('BAY EAGLE', 87.182400000000001), ('GRIFFINITE', 51.217600000000004), ('PERCY HOPE', 48.600000000000001)]
[('MENACING DENNIS', 50.0824), ('MAGIC WEISNER', 40.135300000000001), ('EQUALITY', 24.441199999999998)]
[("FOUFA'S WARRIOR", 18.929399999999998), ('MIDWAY ROAD', 13.823499999999999), ('NEW YORK HERO', 11.188200000000002)]
[('SONG OF THE SWORD', 48.274500000000003), ('WATER CANNON', 36.4529), ('SIR SHACKLETON', 35.088200000000001)]
[('GALLOPING GROCER', 26.4314), ('GOING WILD', 22.911799999999999), ('SUN KING', 19.707800000000002)]
[("GREELEY'S GALAXY", 32.645099999999999), ("HEMINGWAY'S KEY", 25.694099999999999), ('DIABOLICAL', 25.147100000000002)]
[('MINT SLEWLEP', 39.100000000000001), ('C P WEST', 21.076499999999999), ('XCHANGER', 18.666699999999999)]
[('STE